In [ ]:
# shape = feature collection of interest
shape = ee.FeatureCollection(" ")


size = ee.Number(shape.size())
size1 = size.subtract(ee.Number(1))

fc = shape.toList(size)

#date of interest
s = start date
e = end date

dataset = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational').filter(ee.Filter.date(s, e))
total = dataset.reduce(ee.Reducer.sum())
total = total.clip(shape)

stats2 = total.reduceRegions(
  reducer = ee.Reducer.mean(),
  collection = shape,
  scale = 11132,
  )
statsl = ee.List(stats2.toList(size))
def res(m):
  feat = ee.Feature(statsl.get(m))
  id = feat.get('uid')
  f = ee.Feature(shape.filter(ee.Filter.eq('uid', id)).first())
  val = feat.get('hourlyPrecipRate_sum')
  val = ee.Algorithms.If(val, val, 0)
  return f.set(s, val)

mws = ee.List.sequence(0,size1);
l = mws.map(res);
shape = ee.FeatureCollection(l);

task = ee.batch.Export.table.toAsset(**{
  'collection': shape,
  'description': 'Prec_data_fortnight',
  'assetId': ' '
})
task.start()